<a href="https://colab.research.google.com/github/ayulockin/SwAV-TF/blob/master/fine_tuning/Fine_Tuning_10_Epochs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports and Setups

In [ ]:
!git clone https://github.com/ayulockin/SwAV-TF.git

Cloning into 'SwAV-TF'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 204 (delta 3), reused 0 (delta 0), pack-reused 198
Receiving objects: 100% (204/204), 15.83 MiB | 13.22 MiB/s, done.
Resolving deltas: 100% (93/93), done.


In [ ]:
import sys
sys.path.append('SwAV-TF/utils')

import multicrop_dataset
import architecture

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Sequential, Model

import matplotlib.pyplot as plt 
import numpy as np
import random
import time
import os

from tqdm import tqdm
from imutils import paths

tf.random.set_seed(666)
np.random.seed(666)

tfds.disable_progress_bar()

#### W&B - Experiment Tracking

In [ ]:
%%capture
!pip install wandb

In [ ]:
import wandb
from wandb.keras import WandbCallback

wandb.login()

## Restoring model weights from GCS Bucket

In [ ]:
from tensorflow.keras.utils import get_file

In [ ]:
feature_backbone_urlpath = "https://storage.googleapis.com/swav-tf/feature_backbone_10_epochs.h5"
prototype_urlpath = "https://storage.googleapis.com/swav-tf/projection_prototype_10_epochs.h5"

In [ ]:
feature_backbone_weights = get_file('swav_feature_weights', feature_backbone_urlpath)
prototype_weights = get_file('swav_prototype_projection_weights', prototype_urlpath)

17907712/17900192 [==============================] - 1s 0us/step


## Dataset gathering and preparation

We will be using only **10%** labeled data to fine-tune. 

In [ ]:
# Gather Flowers dataset
train_ds, extra_train_ds, validation_ds = tfds.load(
    "tf_flowers",
    split=["train[:10%]", "train[10%:85%]", "train[85%:]"],
    as_supervised=True
)

AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 64

@tf.function
def scale_resize_image(image, label):
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, (224, 224)) # Resizing to highest resolution used while training swav
    return (image, label)

training_ds = (
    train_ds
    .map(scale_resize_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

testing_ds = (
    validation_ds
    .map(scale_resize_image, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead set
data_dir=gs://tfds-data/datasets.



Dataset tf_flowers downloaded and prepared to /root/tensorflow_datasets/tf_flowers/3.0.0. Subsequent calls will reuse this data.


## Get SwAV architecture and Build Linear Model

In [ ]:
def get_linear_classifier():
    # input placeholder
    inputs = Input(shape=(224, 224, 3))
    # get swav baseline model architecture
    feature_backbone = architecture.get_resnet_backbone()
    # load trained weights
    feature_backbone.load_weights(feature_backbone_weights)
    feature_backbone.trainable = False

    x = feature_backbone(inputs, training=False)
    outputs = Dense(5, activation="softmax")(x)
    linear_model = Model(inputs, outputs)

    return linear_model

In [ ]:
tf.keras.backend.clear_session()
model = get_linear_classifier()
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
functional_1 (Functional)    (None, 2048)              23587712  
_________________________________________________________________
dense (Dense)                (None, 5)                 10245     
Total params: 23,597,957
Trainable params: 10,245
Non-trainable params: 23,587,712
_________________________________________________________________


## Callback

In [ ]:
# Early Stopping to prevent overfitting
early_stopper = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=2, verbose=2, restore_best_weights=True)

# Without Augmentation

#### Warm Up

In [ ]:
# get model and compile
tf.keras.backend.clear_session()
model = get_linear_classifier()
model.summary()

model.compile(loss="sparse_categorical_crossentropy", metrics=["acc"],
                     optimizer='adam')

# initialize wandb run
wandb.init(entity='authors', project='swav-tf')

# train
history = model.fit(training_ds,
                 validation_data=(testing_ds),
                 epochs=35,
                 callbacks=[WandbCallback(),
                            early_stopper])

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
functional_1 (Functional)    (None, 2048)              23587712  
_________________________________________________________________
dense (Dense)                (None, 5)                 10245     
Total params: 23,597,957
Trainable params: 10,245
Non-trainable params: 23,587,712
_________________________________________________________________


Epoch 1/35
6/6 [==============================] - 5s 806ms/step - loss: 2.6953 - acc: 0.1608 - val_loss: 1.8782 - val_acc: 0.2364
Epoch 2/35
6/6 [==============================] - 3s 568ms/step - loss: 2.0902 - acc: 0.3106 - val_loss: 1.7566 - val_acc: 0.2436
Epoch 3/35
6/6 [==============================] - 3s 523ms/step - loss: 1.7659 - acc: 0.2561 - val_loss: 1.6900 - val_acc: 0.2909
Epoch 4/35
6/6 [==============================] - 3s 462ms/step - loss: 1.6749 - acc: 0.3324 - val_loss: 1.7713 - val_acc: 0.2600
Epoch 5/35
6/6 [==============================] - 3s 536ms/step - loss: 1.6122 - acc: 0.3460 - val_loss: 1.6062 - val_acc: 0.2891
Epoch 6/35
6/6 [==============================] - 3s 475ms/step - loss: 1.5431 - acc: 0.3624 - val_loss: 1.6708 - val_acc: 0.2764
Epoch 7/35
6/6 [==============================] - 3s 475ms/step - loss: 1.5336 - acc: 0.3624 - val_loss: 1.6525 - val_acc: 0.2764
Epoch 00007: early stopping


In [ ]:
model.save('warmup.h5')

#### Fine tune CNN

In [ ]:
 def get_classifier():
    # input placeholder
    inputs = Input(shape=(224, 224, 3))
    # get swav baseline model architecture
    feature_backbone = architecture.get_resnet_backbone()
    # load trained weights
    feature_backbone.load_weights(feature_backbone_weights)
    feature_backbone.trainable = True

    # load warmup model
    warmup_model = tf.keras.models.load_model('warmup.h5')
    # get trained output layer
    last_layer = warmup_model.get_layer('dense')

    
    x = feature_backbone(inputs, training=False)
    outputs = last_layer(x)
    linear_model = Model(inputs, outputs)

    return linear_model

In [ ]:
# get model and compile
tf.keras.backend.clear_session()
full_trainable_model = get_classifier()
full_trainable_model.summary()

full_trainable_model.compile(loss="sparse_categorical_crossentropy", metrics=["acc"],
                     optimizer='adam')

# initialize wandb
wandb.init(entity='authors', project='swav-tf')

# train                                                        
history = full_trainable_model.fit(training_ds,
                 validation_data=(testing_ds),
                 epochs=35,
                 callbacks=[WandbCallback(),
                            early_stopper])

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
functional_1 (Functional)    (None, 2048)              23587712  
_________________________________________________________________
dense (Dense)                (None, 5)                 10245     
Total params: 23,597,957
Trainable params: 23,544,837
Non-trainable params: 53,120
_________________________________________________________________


Epoch 1/35
      2/Unknown - 1s 304ms/step - loss: 5.0528 - acc: 0.2891WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.2305s vs `on_train_batch_end` time: 0.3763s). Check your callbacks.


6/6 [==============================] - 8s 1s/step - loss: 4.2599 - acc: 0.2807 - val_loss: 2.8384 - val_acc: 0.3291
Epoch 2/35
6/6 [==============================] - 6s 1s/step - loss: 2.0788 - acc: 0.4496 - val_loss: 1.7955 - val_acc: 0.4600
Epoch 3/35
6/6 [==============================] - 6s 1s/step - loss: 1.2194 - acc: 0.5395 - val_loss: 1.4361 - val_acc: 0.5327
Epoch 4/35
6/6 [==============================] - 5s 870ms/step - loss: 0.9144 - acc: 0.6730 - val_loss: 1.6796 - val_acc: 0.5164
Epoch 5/35
6/6 [==============================] - 5s 893ms/step - loss: 0.8932 - acc: 0.6540 - val_loss: 1.6108 - val_acc: 0.5291
Epoch 00005: early stopping


#### Evaluation

In [ ]:
loss, acc = full_trainable_model.evaluate(testing_ds)
wandb.log({'Test Accuracy': round(acc*100, 2)})

9/9 [==============================] - 2s 184ms/step - loss: 1.4361 - acc: 0.5327


# Training with Augmentation


#### Augmentation

In [ ]:
# Configs
CROP_SIZE = 224
MIN_SCALE = 0.5
MAX_SCALE = 1.

# Experimental options
options = tf.data.Options()
options.experimental_optimization.noop_elimination = True
options.experimental_optimization.map_vectorization.enabled = True
options.experimental_optimization.apply_default_optimizations = True
options.experimental_deterministic = False
options.experimental_threading.max_intra_op_parallelism = 1

In [ ]:
@tf.function
def scale_image(image, label):
	image = tf.image.convert_image_dtype(image, tf.float32)
	return (image, label)

@tf.function
def random_apply(func, x, p):
	return tf.cond(
		tf.less(tf.random.uniform([], minval=0, maxval=1, dtype=tf.float32),
				tf.cast(p, tf.float32)),
		lambda: func(x),
		lambda: x)
 
@tf.function
def random_resize_crop(image, label):
  # Conditional resizing
  image = tf.image.resize(image, (260, 260))
  # Get the crop size for given min and max scale
  size = tf.random.uniform(shape=(1,), minval=MIN_SCALE*260,
		          maxval=MAX_SCALE*260, dtype=tf.float32)
  size = tf.cast(size, tf.int32)[0]
  # Get the crop from the image
  crop = tf.image.random_crop(image, (size,size,3))
  crop_resize = tf.image.resize(crop, (CROP_SIZE, CROP_SIZE))
  
  return crop_resize, label

@tf.function
def tie_together(image, label):
  # Scale the pixel values
  image, label = scale_image(image , label)
  # random horizontal flip
  image = random_apply(tf.image.random_flip_left_right, image, p=0.5)
  # Random resized crops
  image, label = random_resize_crop(image, label)
  
  return image, label

In [ ]:
trainloader = (
	train_ds
	.shuffle(1024)
	.map(tie_together, num_parallel_calls=AUTO)
	.batch(BATCH_SIZE)
	.prefetch(AUTO)
)

trainloader = trainloader.with_options(options)

#### Warmup

In [ ]:
# get model and compile
tf.keras.backend.clear_session()
model = get_linear_classifier()
model.summary()

model.compile(loss="sparse_categorical_crossentropy", metrics=["acc"],
                     optimizer='adam')

# initialize wandb run
wandb.init(entity='authors', project='swav-tf')

# train
history = model.fit(trainloader,
                 validation_data=(testing_ds),
                 epochs=35,
                 callbacks=[WandbCallback(),
                            early_stopper])

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
functional_1 (Functional)    (None, 2048)              23587712  
_________________________________________________________________
dense (Dense)                (None, 5)                 10245     
Total params: 23,597,957
Trainable params: 10,245
Non-trainable params: 23,587,712
_________________________________________________________________


Epoch 1/35
6/6 [==============================] - 4s 677ms/step - loss: 2.5339 - acc: 0.2125 - val_loss: 2.2990 - val_acc: 0.2600
Epoch 2/35
6/6 [==============================] - 4s 591ms/step - loss: 1.8990 - acc: 0.3270 - val_loss: 1.8746 - val_acc: 0.2618
Epoch 3/35
6/6 [==============================] - 4s 609ms/step - loss: 1.7228 - acc: 0.3324 - val_loss: 1.6946 - val_acc: 0.3164
Epoch 4/35
6/6 [==============================] - 3s 583ms/step - loss: 1.6566 - acc: 0.3842 - val_loss: 1.6715 - val_acc: 0.2618
Epoch 5/35
6/6 [==============================] - 4s 626ms/step - loss: 1.5973 - acc: 0.3215 - val_loss: 1.6127 - val_acc: 0.2745
Epoch 6/35
6/6 [==============================] - 3s 562ms/step - loss: 1.5181 - acc: 0.3624 - val_loss: 1.5963 - val_acc: 0.2782
Epoch 7/35
6/6 [==============================] - 3s 531ms/step - loss: 1.5553 - acc: 0.3488 - val_loss: 1.6605 - val_acc: 0.3055
Epoch 8/35
6/6 [==============================] - 3s 559ms/step - loss: 1.5332 - acc: 0.34

In [ ]:
model.save('warmup_augmentation.h5')

#### Fine tune CNN

In [ ]:
 def get_classifier():
    # input placeholder
    inputs = Input(shape=(224, 224, 3))
    # get swav baseline model architecture
    feature_backbone = architecture.get_resnet_backbone()
    # load trained weights
    feature_backbone.load_weights(feature_backbone_weights)
    feature_backbone.trainable = True

    # load warmup model
    warmup_model = tf.keras.models.load_model('warmup_augmentation.h5')
    # get trained output layer
    last_layer = warmup_model.get_layer('dense')

    
    x = feature_backbone(inputs, training=False)
    outputs = last_layer(x)
    linear_model = Model(inputs, outputs)

    return linear_model

In [ ]:
# get model and compile
tf.keras.backend.clear_session()
full_trainable_model = get_classifier()
full_trainable_model.summary()

full_trainable_model.compile(loss="sparse_categorical_crossentropy", metrics=["acc"],
                     optimizer='adam')

# initialize wandb
wandb.init(entity='authors', project='swav-tf')

# train                                                        
history = full_trainable_model.fit(trainloader,
                 validation_data=(testing_ds),
                 epochs=35,
                 callbacks=[WandbCallback(),
                            early_stopper])

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
functional_1 (Functional)    (None, 2048)              23587712  
_________________________________________________________________
dense (Dense)                (None, 5)                 10245     
Total params: 23,597,957
Trainable params: 23,544,837
Non-trainable params: 53,120
_________________________________________________________________


Epoch 1/35
      2/Unknown - 1s 302ms/step - loss: 5.1523 - acc: 0.1953WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.2202s vs `on_train_batch_end` time: 0.3816s). Check your callbacks.


6/6 [==============================] - 7s 1s/step - loss: 4.6954 - acc: 0.2480 - val_loss: 2.2529 - val_acc: 0.3327
Epoch 2/35
6/6 [==============================] - 6s 1s/step - loss: 1.6760 - acc: 0.4741 - val_loss: 1.4216 - val_acc: 0.4655
Epoch 3/35
6/6 [==============================] - 5s 867ms/step - loss: 1.2600 - acc: 0.5286 - val_loss: 1.5094 - val_acc: 0.4364
Epoch 4/35
6/6 [==============================] - 6s 1s/step - loss: 1.2839 - acc: 0.5123 - val_loss: 1.3354 - val_acc: 0.5200
Epoch 5/35
6/6 [==============================] - 5s 872ms/step - loss: 1.1615 - acc: 0.5477 - val_loss: 1.4144 - val_acc: 0.5036
Epoch 6/35
6/6 [==============================] - 5s 882ms/step - loss: 1.1305 - acc: 0.5150 - val_loss: 1.3485 - val_acc: 0.5145
Epoch 00006: early stopping


#### Evaluation

In [ ]:
loss, acc = full_trainable_model.evaluate(testing_ds)
wandb.log({'Test Accuracy': round(acc*100, 2)})

9/9 [==============================] - 2s 188ms/step - loss: 1.3354 - acc: 0.5200
